In [1]:
import pandas as pd
import numpy as np
import json

**DJIA**

In [2]:
df = pd.read_csv('data/DJIA_data.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2006-12-28,12510.570313,12529.879883,12478.129883,12501.519531,12501.519531,126740000
1,2006-12-29,12500.480469,12526.030273,12451.129883,12463.150391,12463.150391,161560000
2,2007-01-03,12459.540039,12580.349609,12404.820313,12474.519531,12474.519531,327200000
3,2007-01-04,12473.160156,12510.410156,12403.860352,12480.690430,12480.690430,259060000
4,2007-01-05,12480.049805,12480.129883,12365.410156,12398.009766,12398.009766,235220000


In [3]:
df = df[['Date', 'Close','Adj Close']]
df.Date = pd.to_datetime(df.Date)
df = df.set_index('Date')
df.head()

,Close,Adj Close
Date,,
2006-12-28,12501.519531,12501.519531
2006-12-29,12463.150391,12463.150391
2007-01-03,12474.519531,12474.519531
2007-01-04,12480.690430,12480.690430
2007-01-05,12398.009766,12398.009766


In [4]:
idx = pd.date_range('2006-12-28', '2016-12-31')
df = df.reindex(idx, fill_value=np.NaN) # fill_value : reindex 과정에서 생긴 결측값 채우기
df.head()

,Close,Adj Close
2006-12-28,12501.519531,12501.519531
2006-12-29,12463.150391,12463.150391
2006-12-30,NaN,NaN
2006-12-31,NaN,NaN
2007-01-01,NaN,NaN


### 결측값 보간
시계열 데이터 분석이라든지 이미지 분석 등에서 사용하면 매우 유용하고 편리한 method 입니다.
이전 포스팅의 결측값 대체는 '특정의 동일 값'으로 채우는 방식(filling, imputation)이었던 반면에, 이번 포스팅의 결측값 보간(interploation)은 실측값과 실측값 사이의 결측값을 마치 '그라데이션(gradation)' 기법으로 색깔을 조금씩 변화시켜가면서 부드럽게 채워나가는 방법
<img src="https://t1.daumcdn.net/cfile/tistory/27380C3F584C23A617" />
참고 : https://rfriend.tistory.com/264

In [5]:
df = df.interpolate() # interpolate() : 결측값을 보간
df.head()

,Close,Adj Close
2006-12-28,12501.519531,12501.519531
2006-12-29,12463.150391,12463.150391
2006-12-30,12465.424219,12465.424219
2006-12-31,12467.698047,12467.698047
2007-01-01,12469.971875,12469.971875


**NYT**

In [13]:
years = range(2007, 2016)
months = range(1,13)
dict_keys = ['type_of_material','headline.main','pub_date','section_name','news_desk']
s_list = ['business', 'national', 'world', 'u.s.' , 'politics', 'opinion', 'tech', 'science',  'health']

NYT_final = pd.DataFrame()
for year in years:
    for month in months:
        file_str = 'data/nytime_json/nytimes'+str(year) + '-' + '{:02}'.format(month) + '.json'
        with open(file_str) as f:
            NYT = json.load(f)
        NYT = NYT['response']['docs']
        main_df = pd.DataFrame()
        for article in NYT:
            temp = pd.io.json.json_normalize(article)
            temp = temp.filter(items=dict_keys)
            main_df = pd.concat([main_df, temp])
        main_df.pub_date = pd.to_datetime(main_df.pub_date).dt.date
        main_df.section_name = main_df.section_name.fillna('UNK').str.lower()
        main_df = main_df[main_df.section_name.str.contains('|'.join(s_list))]
        main_df = main_df.set_index('pub_date')
        NYT_final = pd.concat([NYT_final,main_df])

C:\Users\kduar_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\kduar_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



KeyboardInterrupt: 

In [6]:
NYT_final.to_pickle('data/pickled_NYT.pkl')

NameError: name 'NYT_final' is not defined

In [14]:
headline = NYT_final.groupby(NYT_final.index)['headline.main'].agg(lambda x: ' '.join(x.astype('str'))).reset_index()
headline.head()

,pub_date,headline.main
0,2007-01-01,Estimates of Iraqi Civilian Deaths Romania and...
1,2007-01-02,"For Dodd, Wall Street Looms Large Ford's Lost ..."
2,2007-01-03,"Ethics Changes Proposed for House Trips, K Str..."
3,2007-01-04,I Feel Bad About My Face Bush Recycles the Tra...
4,2007-01-05,Macworld Bingo Anti-Surge Protests Against McC...


In [9]:
NYT_df = pd.read_pickle('data/pickled_NYT.pkl')
NYT_df.head()

,headline.main,news_desk,section_name,type_of_material
pub_date,,,,
2007-01-01,Estimates of Iraqi Civilian Deaths,National,u.s.,News
2007-01-01,Romania and Bulgaria Join European Union,ContinuousNews,world,News
2007-01-01,Supporters of Thai Ex-Premier Blamed for Blasts,ContinuousNews,world,News
2007-01-01,U.S. Questioned Iraq on the Rush to Hang Hussein,Foreign,world,News
2007-01-01,States Take Lead on Ethics Rules for Lawmakers,National,u.s.,News


In [10]:
NYT_df.shape

(525542, 4)

In [11]:
headline1 = NYT_df.groupby(NYT_df.index)['headline.main'].agg(lambda x: ' '.join(x.astype('str'))).reset_index()
headline1.head()

,pub_date,headline.main
0,2007-01-01,Estimates of Iraqi Civilian Deaths Romania and...
1,2007-01-02,"For Dodd, Wall Street Looms Large Ford's Lost ..."
2,2007-01-03,"Ethics Changes Proposed for House Trips, K Str..."
3,2007-01-04,I Feel Bad About My Face Bush Recycles the Tra...
4,2007-01-05,Macworld Bingo Anti-Surge Protests Against McC...


In [12]:
df.head()

,Close,Adj Close
2006-12-28,12501.519531,12501.519531
2006-12-29,12463.150391,12463.150391
2006-12-30,12465.424219,12465.424219
2006-12-31,12467.698047,12467.698047
2007-01-01,12469.971875,12469.971875


### data/raw_sample.pkl 보완

In [13]:
df1 = pd.read_csv('data/DJIA_data.csv')
#df1 = df[['Date', 'Close','Adj Close']]
df1.Date = pd.to_datetime(df1.Date)
df1 = df1.set_index('Date')
df1.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2006-12-28,12510.570313,12529.879883,12478.129883,12501.519531,12501.519531,126740000
2006-12-29,12500.480469,12526.030273,12451.129883,12463.150391,12463.150391,161560000
2007-01-03,12459.540039,12580.349609,12404.820313,12474.519531,12474.519531,327200000
2007-01-04,12473.160156,12510.410156,12403.860352,12480.690430,12480.690430,259060000
2007-01-05,12480.049805,12480.129883,12365.410156,12398.009766,12398.009766,235220000


In [14]:
idx = pd.date_range('2006-12-28', '2016-12-31')
df1 = df1.reindex(idx, fill_value=np.NaN) # fill_value : reindex 과정에서 생긴 결측값 채우기
df1.head()

,Open,High,Low,Close,Adj Close,Volume
2006-12-28,12510.570313,12529.879883,12478.129883,12501.519531,12501.519531,126740000.0
2006-12-29,12500.480469,12526.030273,12451.129883,12463.150391,12463.150391,161560000.0
2006-12-30,NaN,NaN,NaN,NaN,NaN,NaN
2006-12-31,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-01,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df1 = df1.interpolate() # interpolate() : 결측값을 보간
df1.head()

,Open,High,Low,Close,Adj Close,Volume
2006-12-28,12510.570313,12529.879883,12478.129883,12501.519531,12501.519531,126740000.0
2006-12-29,12500.480469,12526.030273,12451.129883,12463.150391,12463.150391,161560000.0
2006-12-30,12492.292383,12536.894140,12441.867969,12465.424219,12465.424219,194688000.0
2006-12-31,12484.104297,12547.758007,12432.606055,12467.698047,12467.698047,227816000.0
2007-01-01,12475.916211,12558.621875,12423.344141,12469.971875,12469.971875,260944000.0


In [17]:
df_KDU = df1.join(headline1.set_index('pub_date')).dropna()
df_KDU.to_pickle('data/raw_sample_KDU.pkl')

In [9]:
df2 = df.join(headline.set_index('pub_date')).dropna()
df2.to_pickle('data/raw_sample.pkl')